## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Вывод

В результате изучения общей информации видно, что есть пропуски в столбцах days_employed и total_income. Так же тип значений в этих столбцах float64. Для ускорения обработки и прощения восприятия заменим тип на int64.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
# Методами ".isnull().sum()"  и ".value_counts()" выявил пропущенные значения и опечатки в данных
# Решил не удалять 2174 строк с нулевыми значениями т.к. среди них 170 должников
# Чтобы ипользовать данные о детях и семейном положении из столбцов с пропусками, заменим пропущенные значения на медиану.
# Медиану выбираем, потому что в столбце есть слишком большие и отрицательные значения в столбце days_employed.
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)#заполнили пробела медианой в стаже
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(total_income_median)#заполнили пробела медианой в доходе
print(data.isnull().sum()) 

# Исправляем опечатки в столбце ДЕТИ. 20 на 2, а -1 на 1.
data['children'] = data.replace(data[data['children']==20], 2)
data['children'] = data.replace(data[data['children']==-1], 1)
print(data['children'].value_counts())


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


# Вывод

После заполнения пропусков и исправления опечаток можно приступить к дальнейшей обработке данных и ничего не упустить.

<div class="alert alert-block alert-warning">
То, что ты решил сделать замену пропущенных данных - похвально. Но заменять общей медианной слишком грубо. Куда корректнее будет заменить медианой в зависимости от других параметров (например, образование, семейное положение и т.д.)
</div>
    
---

### Замена типа данных

In [ ]:
#переводим значения столбцов стажа и дохода из float в int64 методом .astype()
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null object
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


### Вывод

Теперь видим что в данных нет пропусков и только два типа данных

<div class="alert alert-block alert-success">
Здесь всё хорошо
</div>
    
---

### Обработка дубликатов

In [ ]:

data['education'] = data['education'].str.lower() # привел значения к нижнему регистру
display(data['education'].value_counts()) 
# дубликаты появились из-за заполнения формы используя верхний и нижний регистр, программа распознает их как разные слова

#решил не объединять похожие категории в стаусе из-за возможно разных значений корреляции по ним
display(data['family_status'].value_counts())


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Вывод

После приведения к нижнему регистру столбца с образованием оказалось, что там всего 5 категорий.

# Лемматизация

In [ ]:
import pandas as pd
from pymystem3 import Mystem  # подгрузил словарь
from collections import Counter
m = Mystem() 
data = pd.read_csv('/datasets/data.csv')
# Создал функцию, которая опрашивает содержание столбца "Цель кредита" после лемматизации
# И возвращает в него ключевое значение
def lemmatize(row):    
    lemmas = m.lemmatize(row)    
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif 'недвижимость'  in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    return 'недвижимость'
data['purpose'] = data['purpose'].apply(lemmatize)

print(data['purpose'].value_counts())

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose, dtype: int64


### Вывод

После обработки столбца "цель кредита" выделил 4-е ключевые причины кредитования.

<div class="alert alert-block alert-success">
Ты выделил основные цели кредита. Хорошо
</div>
    
---

### Категоризация данных

In [ ]:
# Создадим таблицу, которая показывает факт возврата долга в зависимости от количества детей
data['children'] = data.replace(data[data['children']==20], 2) 
data['children'] = data.replace(data[data['children']==-1], 1)
data_children = data.pivot_table(index='children', columns='debt', values='family_status',aggfunc='count')
# Создадим таблицу, которая показывает факт возврата долга в зависимости от семейного положения
data_status = data.pivot_table(index='family_status', columns='debt', values='children',aggfunc='count')
display(data_children)
display(data_status)


debt,0,1
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1929.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


debt,0,1
family_status,,
Не женат / не замужем,2539,274
в разводе,1110,85
вдовец / вдова,897,63
гражданский брак,3789,388
женат / замужем,11449,931


### Вывод

Сформировал две таблицы из которых видна зависимость между возвратом кредита в срок, количеством детей и статуса заемщиков.

<div class="alert alert-block alert-warning">
Для вывода датафрейм объектов лучше использовать display(), а не print()
</div>
    
---

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#рассчитаем конверсию должников в зависимости от наличия детей
data['children'] = data.replace(data[data['children']==20], 2)
data['children'] = data.replace(data[data['children']==-1], 1)
data_children = data.pivot_table(index='children', columns='debt', values='family_status_id',aggfunc='count')
data_children['conversion'] = data_children[1] / (data_children[0] + data_children[1])
print(data_children)


debt            0       1  conversion
children                             
0         13086.0  1063.0    0.075129
1          4420.0   445.0    0.091470
2          1929.0   202.0    0.094791
3           303.0    27.0    0.081818
4            37.0     4.0    0.097561
5             9.0     NaN         NaN


# Вывод

Из соотношения числа должников к общему числу в каждой группе видим, что заемщики с детьми хуже возвращают кредит, чем заемщики без детей.

<div class="alert alert-block alert-warning">
Понял твою идею, но не совсем ясно какой порог берешь. Какое значение должно быть в разнице конверсий, чтобы можно было сказать о зависимости
</div>
    
---

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_status = data.pivot_table(index='family_status', columns='debt', values='children',aggfunc='count')
data_status['conversion'] = data_status[1] / (data_status[0] + data_status[1])
print(data_status)

debt                       0    1  conversion
family_status                                
Не женат / не замужем   2539  274    0.097405
в разводе               1110   85    0.071130
вдовец / вдова           897   63    0.065625
гражданский брак        3789  388    0.092890
женат / замужем        11449  931    0.075202


### Вывод

Не женатые/не замужем и проживающие в гражданском браке хуже возвращают кредит в срок, чем женатые, в разводе или вдовцы.
Вдовы/вдовцы самые дисциплинированные.

<div class="alert alert-block alert-warning">
Для наглядности, я бы тогда отсортировал по значению, чтобы сразу было видно топ.
</div>
    
---

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data['total_income'].quantile([.25, .5, .75])
def income_level(row):      
    if row<=103053.152913:
        return 'низкий'    
    if (row>103053.15291) and (row<=145017.937533):
        return 'ниже среднего'
    if (row>145017.937533) and (row<=203435.067663):
        return 'выше среднего'
    else:
        return 'высокий'
    
data['total_income_level'] = data['total_income'].apply(income_level)
data_income = data.pivot_table(index=str('total_income_level'), columns='debt', values='children',aggfunc='count')
data_income['conversion'] = data_income[1] / (data_income[0] + data_income[1])
print(data_income)

debt                   0    1  conversion
total_income_level                       
высокий             6501  511    0.072875
выше среднего       4411  426    0.088071
ниже среднего       4417  421    0.087019
низкий              4455  383    0.079165


### Вывод

Заемщиков с высоким доходом больше всех и они самые дисциплинированные. 
Интересно, что на втором месте, среди тех кто возвращает долг в срок, заемщики с самым низким доходом

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_purpose = data.pivot_table(index='purpose', columns='debt', values='family_status_id',aggfunc='count')
data_purpose['conversion'] = data_purpose[1] / (data_purpose[0] + data_purpose[1])
print(data_purpose)

debt              0    1  conversion
purpose                             
автомобиль     3912  403    0.093395
недвижимость  10058  782    0.072140
образование    3652  370    0.091994
свадьба        2162  186    0.079216


### Вывод

Хуже всего возвращают кредит на автомобиль и образование. Лучше дела с кредитом на свадьбу и покупку недвижимости. 
Покупатели недвижимости самые дисциплинированные. С покупателями авто нужно быть осторожными.

### Шаг 4. Общий вывод

Самая лучшая выборочная группа - это женатые(замужем) заемщики без детей и с высоким доходом, которые берут кредит на преобретение недвижимости. И по статистике, банк больше всего дает кредит заемщикам с такими характеристиками.